In [1]:
import os
import time
import json
from datetime import datetime
from pathlib import Path
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup, SoupStrainer

In [2]:
base = 'scraped2'
home = base + '/www/hochschule-rhein-waal/de'

skip = ["https://www.hochschule-rhein-waal.de/de/fakultaeten/life-sciences/https://www.hochschule-rhein-waal.de/en/faculties/life-sciences/degree-programmes/bio-science-and-health-bsc/bio-science-and-health-bsc",
        "https://www.hochschule-rhein-waal.de"
        ]

path = Path(base)
path.mkdir(parents=True, exist_ok=True)

visited = []

writeLater = []

badstart = ("https://www.hochschule-rhein-waal.de/hochschule",
            
            "https://www.hochschule-rhein-waal.de/de/user",
            "https://www.hochschule-rhein-waal.de/en/user",

            "https://www.hochschule-rhein-waal.de/de/system",
            "https://www.hochschule-rhein-waal.de/en/system",

            "https://www.hochschule-rhein-waal.de/de/node",
            "https://www.hochschule-rhein-waal.de/en/node",
            
            "https://www.hochschule-rhein-waal.de/de/search",
            "https://www.hochschule-rhein-waal.de/en/search",

            "https://www.hochschule-rhein-waal.de/de/webform",
            "https://www.hochschule-rhein-waal.de/en/webform",
            
            "https://www.hochschule-rhein-waal.de/de/webformular",
            "https://www.hochschule-rhein-waal.de/en/webformular",

            "https://www.hochschule-rhein-waal.de/de/homepage",
            "https://www.hochschule-rhein-waal.de/en/homepage",

            "https://www.hochschule-rhein-waal.de/de/event",
            "https://www.hochschule-rhein-waal.de/en/event",

            "https://www.hochschule-rhein-waal.de/de/panel",
            "https://www.hochschule-rhein-waal.de/en/panel",

            "https://www.hochschule-rhein-waal.de/de/news",
            "https://www.hochschule-rhein-waal.de/en/news",

            "https://www.hochschule-rhein-waal.de/en/fakultaeten",

            "https://www.hochschule-rhein-waal.de/de/aktuelles",

            "https://www.hochschule-rhein-waal.de/file",

            "https://www.hochschule-rhein-waal.de/media",

            "https://www.hochschule-rhein-waal.de/node",
            
            "https://www.hochschule-rhein-waal.de/sites/default/files/images",

            "https://www.hochschule-rhein-waal.de/sites/default/files/audios",

            "https://www.hochschule-rhein-waal.de/sites/default/files/videos",

            "https://www.hochschule-rhein-waal.de/sites/default/files/styles",

            "https://www.hochschule-rhein-waal.de/faculties",

            "https://www.hochschule-rhein-waal.de/seite",

            "https://www.hochschule-rhein-waal.de/system",

            "https://www.hochschule-rhein-waal.de/twitter",

            "https://www.hochschule-rhein-waal.de/user",

            "https://www.hochschule-rhein-waal.de/www",

            "https://www.hochschule-rhein-waal.de//www",

            "https://www.hochschule-rhein-waal.de/youtube",

            "https://www.hochschule-rhein-waal.de//youtube",

            "https://www.hochschule-rhein-waal.de/nl",

            "https://www.hochschule-rhein-waal.de/de?field",
            "https://www.hochschule-rhein-waal.de/en?field",
            "https://www.hochschule-rhein-waal.de?field",
            )

bad = ["http:", "@", "detel", "his.", ".jpg", ".png", ".mp4", ".docx", "?page=", '\\"', "img src", ".pdf .html", ".pdf.html"]

# function to check if its a full link or just relative
def isAbsoluteLink(url):
    parse = urlparse(url)
    return parse.scheme != '' and parse.netloc != ''

# function to check if its hsrw website url
def isHsrwURL(url):
    return urlparse(url).netloc == 'www.hochschule-rhein-waal.de'

# function to turn relative path to hsrw website
def toHsrwURL(relative):
    return "https://www.hochschule-rhein-waal.de" + relative

# function to create dir path
def urlToDirPath(_URL):
    url = "/".join(_URL.split("/")[3:-1])
    parse = urlparse(url)
    return Path(base , str(parse.netloc).replace('.','/') + parse.path , parse.query)

def fullUrlToDirPath(_URL):
    url = "/".join(_URL.split("/")[3:])
    parse = urlparse(url)
    return Path(base , str(parse.netloc).replace('.','/') + parse.path , parse.query)

def fileExist(_URL):
    path = urlToDirPath(_URL)
    #path.mkdir(parents=True, exist_ok=True)
    if _URL.endswith((".pdf", "attachment", "attachment=")):
        fileName = _URL.split("/")[-1]
        if not fileName.endswith(".pdf"):
            fileName = str(path).split('\\')[-1] + ".pdf"
    else:
        fileName = _URL.split("/")[-1] + ".html"
        special = ["?", ":"]
        fileName = ''.join(c for c in fileName if c not in special)

    fullPath = Path.joinpath(path , fileName)
    if os.path.isfile(fullPath):
        return fullPath

def fileExist2(filePath):
    if os.path.isfile(filePath):
        return filePath

# function to write file to path
def writeHTMLToPath(soup, _URL):
    parse = urlparse(_URL)
    if not parse.path:
        return

    path = urlToDirPath(_URL)
    path.mkdir(parents=True, exist_ok=True)

    fileName = _URL.split("/")[-1] + ".html"

    special = ["?", ":"]

    fileName = ''.join(c for c in fileName if c not in special)

    fullPath = Path.joinpath(path , fileName)
    with open(fullPath, "w") as file:
        file.write(str(soup))
    writeMetaData(path, fileName, fullPath, _URL, getTitle(soup), "HTML")

def writePDFToPath(pdf, _URL, origin, _title):
    global counter

    path = fullUrlToDirPath(origin)
    path.mkdir(parents=True, exist_ok=True)

    path2 = urlToDirPath(_URL)

    fileName = _URL.split("/")[-1]
    if fileName == "f_provide_evidence_of_illness-related_inability_to_participate_in_examinations_medical_attestations_for_presentation_to_the_examination_service.pdf":
        fileName = "illness-related_inability_participate_examinations.pdf"
    if not fileName.endswith(".pdf"):
        fileName = str(path2).split('\\')[-1] + ".pdf"

    fullPath = Path.joinpath(path , fileName)
    with open(fullPath, "wb") as file:
        file.write(pdf)

    writeMetaData(path, fileName, fullPath, origin, _title, "PDF", _URL)
    
def getTitle(_soup):
    title_tag = _soup.title
    return title_tag.string.strip() if title_tag and title_tag.string else None

def getText(_element):
    if _element:
        return _element.get_text(strip=True)
    return None

def writeMetaData(path, fileName, filePath, source_url, title, document_type, download_url = ""):
    metadataFile = Path.joinpath(path , fileName + ".metadata.json")
    metadata = {
        "file_path": str(filePath),
        "title": title,
        "source_url": source_url,
        "download_url": download_url,
        "document_type": document_type,
        "scraped_at": datetime.today().strftime('%Y-%m-%d')
    }
    with open(metadataFile, "w") as file:
        json.dump(metadata, file, indent=4)
#counter
counter = 0

#request function
def make_request(URL):
    URL = URL.strip()
    global counter

    if URL.endswith('\\'):
        URL = URL[:-1]

    next = []
    links = []
    
    if URL.endswith((".pdf", "attachment", "attachment=", ".jpg", ".png")) or URL in skip:
        print("SOMETHING THAT SHOULD NOT HAPPEN")
    else:
        soup = None
        if fileExist(URL):
            print("Starting page from Disk:", URL, URL in visited, "counter:", counter)
            with open(fileExist(URL)) as fp:
                soup = BeautifulSoup(fp, 'html.parser')
        else:
            print("Starting page from Website:", URL, URL in visited, "counter:", counter)
            page = requests.get(URL)
            time.sleep(0.4)
            soup = BeautifulSoup(page.content, "html.parser")
        if URL not in visited:
            writeHTMLToPath(soup, URL)
            counter += 1
            visited.append(URL)
        #find all links on page from URL
        for a in soup.find_all('a', href=True):
            link = a['href'].strip()
            if not link.startswith('#') and link != '':
                if link.endswith(("\\")):
                    link = link[:-1]
                if not isAbsoluteLink(link):
                    link = toHsrwURL(link)
                if link not in visited and link not in skip and isHsrwURL(link) and not any(b in link for b in bad) and not link.startswith((badstart)):
                    _tmp = {"link":link, "text": getText(a)}
                    _skip = False
                    for l in links:
                        if l["link"]==link:
                            _skip = True
                            if len(l["text"]) < len(_tmp["text"]):
                                links.remove(l)
                                links.append(_tmp)
                    if not _skip:        
                        links.append(_tmp)
        
    # visit each link and scrape it, 
    for _entry in links:
        link = _entry["link"]
        if link not in visited:
            visited.append(link)
            counter += 1

            if fileExist(link):
                if link.endswith((".pdf", "attachment", "attachment=")):
                    print("PDF File already exists on Disk:", link, "counter:", counter)
                else:
                    print("HTML File already exists on Disk:", link, "counter:", counter)
                    if isHsrwURL(link):
                            next.append(link)
            else:
                page = requests.get(link)
                if link.endswith((".pdf", "attachment", "attachment=")):
                    print("PDF File Scraping from Website:", link, "counter:", counter)
                    if _entry["text"]:
                        writePDFToPath(page.content, link, URL, getTitle(soup) + ' - ' + _entry["text"])
                    else:
                        tmp = {
                            "content": page.content,
                            "link": link,
                            "url": URL,
                            "title_summary": getTitle(soup) + ' - ' + _entry["text"]
                        }
                        if tmp not in writeLater:
                            writeLater.append(tmp)
                else:
                    print("HTML File Scraping from Website:", link, "counter:", counter)
                    soup = BeautifulSoup(page.content, "html.parser")
                    writeHTMLToPath(soup, link)
                    if isHsrwURL(link):
                        next.append(link)
                time.sleep(0.4)

    print("===========")
    print("visited", len(visited), "in total")
    print("===========")
    # do the same with links in next
    for link in next:
        make_request(link)

In [4]:
URLS = ["https://www.hochschule-rhein-waal.de/de"]

for url in URLS:
    make_request(url)
print("FINISHED SCRAPING")

Starting page from Website: https://www.hochschule-rhein-waal.de/de False counter: 0
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/en counter: 2
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/presse counter: 3
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/studieninteressierte counter: 4
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/studierende counter: 5
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/wirtschaft counter: 6
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/jobs counter: 7
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/hochschule counter: 8
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/studium counter: 9
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/de/fakultaeten counter: 10
HTML File Scraping from Website: https://www.hochschule-rhein-waal.de/d